In [1]:
# 신경망 정의하기

import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    # 입력 이미지 채널 1개, 출력 채널 6개, 3X3의 정사각 컨볼루션 행렬
    # 컨볼루션 커널 정의
    self.conv1 = nn.Conv2d(1, 6, 3)
    self.conv2 = nn.Conv2d(6, 16, 3)
    
    # 아핀(affine) 연산: y = W x + b
    self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6 * 6은 이미지 차원에 해당
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    # (2, 2) 크기 윈도우에 대해 맥스 풀링(max pooling)
    x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))

    #크기가 제곱수라면 하나의 숫자만을 특정
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  
  def num_flat_features(self, x):
    size = x.size()[1:] # 배치 차원을 제외한 모든 차원
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1의 .weight

10
torch.Size([6, 1, 3, 3])


In [3]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0050,  0.1108, -0.0455,  0.0112, -0.0918,  0.0984,  0.0201, -0.0802,
          0.0168, -0.0865]], grad_fn=<AddmmBackward>)


In [4]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [6]:
# 손실 함수
output = net(input)
target = torch.randn(10) # 예시를 위한 임의의 정답
target = target.view(1,-1) # 출력과 같은 shape로 만듦
print(target)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor([[ 0.3932, -0.8892, -0.4863,  1.0519,  0.7324,  0.9881,  0.3326, -0.0286,
         -1.4036,  0.1234]])
tensor(0.6060, grad_fn=<MseLossBackward>)


In [8]:
print(loss.grad_fn) # MSE Loss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

In [9]:
# 역전파
net.zero_grad() # 모든 매개변수의 변화도 버퍼를 0으로 만듬

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0128, -0.0051, -0.0091,  0.0077, -0.0096, -0.0032])


In [10]:
# 가중치 갱신
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

In [11]:
import torch.optim as optim

# Optimizer를 생성합니다.
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)에서는 다음과 같습니다:
optimizer.zero_grad() # 변화도 버퍼를 0으로
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # 업데이트 진행